In [1]:
import pandas as pd
import os
from pathlib import Path

DATA_DIR = Path(os.path.abspath('')).parents[1] / "data"
STOCK_PRICES_DIR_RAW = DATA_DIR / "stocks" / "raw"
TRANSPORT_FARES_DIR = DATA_DIR / "fares"
TRANSPORT_FARES_DIR

PosixPath('/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/fares')

# Transport Fares

In [10]:
bus_and_coach_fares = pd.read_csv(TRANSPORT_FARES_DIR / "raw" / "bus_and_coach_fares.csv", skiprows=7)
rail_fares = pd.read_csv(TRANSPORT_FARES_DIR / "raw" / "rail_fares.csv", skiprows=7)
tfl_fares = pd.read_csv(TRANSPORT_FARES_DIR / "raw" / "TfLHistoricalFares2000to2025.csv")

In [11]:
rail_fares

,Year,relative_price
0,1987,100.6
1,1988,107.6
2,1989,117.4
3,1990,127.7
4,1991,141.0
...,...,...
642,2024 SEP,515.0
643,2024 OCT,523.4
644,2024 NOV,519.5
645,2024 DEC,519.6


# Stock Indeces

In [45]:
def preprocess_stock_prices_dataset(fpath: Path) -> None:
    df = pd.read_csv(fpath)
    df["Date"] = pd.to_datetime(df["Date"])
    df["year"] = df.Date.dt.year

    df_processed = df.groupby("year").max().sort_values("year")
    df_processed.columns = [s.strip().lower() for s in df_processed.columns]

    if ("year" not in df_processed.columns):
        df_processed = df_processed.reset_index(drop=False)

    # df_processed.to_csv(fname[:-4] + "_processed.csv", index=False)
    new_fname =  fpath.name[:-4] + "_processed.csv"
    df_processed.to_csv(fpath.parents[1] / "processed" / new_fname, index=False)

In [46]:
for fpath in STOCK_PRICES_DIR_RAW.iterdir():
    if ("csv" in str(fpath)):
        print(fpath.parent, fpath.name)
        preprocess_stock_prices_dataset(fpath)

/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw HangSeng_1985to2025.csv
/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw FTSE100_1985to2025.csv


/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_57265/1996819000.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])
/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_57265/1996819000.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw Nikkei225_1980to2025.csv
/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw SnP500_1980to2025.csv


/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_57265/1996819000.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])
/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_57265/1996819000.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


/Users/avekassy/Desktop/codingProjects/visualisations/tfl-price-visualisations/data/stocks/raw DAX_1987to2025.csv


/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_57265/1996819000.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["Date"] = pd.to_datetime(df["Date"])


In [ ]:
snp = pd.read_csv(STOCK_PRICES_DIR_RAW / "SnP500_1980to2025.csv")
snp["Date"] = pd.to_datetime(snp["Date"])
snp["year"] = snp.Date.dt.year
snp.head()

/var/folders/1p/qx30w4js39sbtnrknydp18zc0000gn/T/ipykernel_57265/1726749124.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  snp["Date"] = pd.to_datetime(snp["Date"])


,Date,Open,High,Low,Close,year
0,2025-09-05,6529.08,6532.65,6443.98,6481.50,2025
1,2025-09-04,6456.60,6502.54,6445.98,6502.08,2025
2,2025-09-03,6445.82,6453.67,6416.17,6448.26,2025
3,2025-09-02,6401.51,6416.54,6360.58,6415.54,2025
4,2025-08-29,6489.28,6491.76,6444.57,6460.26,2025


In [ ]:
snp_processed = snp.groupby("year").max().sort_values("year")
snp_processed.columns = [s.strip().lower() for s in snp_processed.columns]

In [11]:
if ("year" not in snp_processed.columns):
    snp_processed = snp_processed.reset_index(drop=False)
snp_processed

,year,date,open,high,low,close
0,1980,1980-12-31,140.52,141.96,139.00,140.52
1,1981,1981-12-31,138.12,140.32,135.86,138.12
2,1982,1982-12-31,143.02,144.65,143.11,143.02
3,1983,1983-12-30,172.65,173.10,169.65,172.65
4,1984,1984-12-31,170.41,170.41,168.66,170.41
5,1985,1985-12-31,212.02,213.02,210.78,212.02
6,1986,1986-12-31,254.00,254.85,253.35,254.00
7,1987,1987-12-31,336.77,337.89,334.46,336.77
8,1988,1988-12-30,283.66,283.95,282.28,283.66
9,1989,1989-12-29,359.80,360.44,358.11,359.80


In [ ]:
# It would make a lot of sense to combine them into one csv
# TODO